## Preprocess

In [16]:
import mido
from mido import Message, MidiFile, MidiTrack,MetaMessage
import string
import numpy as np
import pandas as pd
from midi_array import *
import os
import csv

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
# label path     
label_path = r'D:\BrownUnivercity\CS2470\final_proj\CS2470_final_project\data\label.csv'
# Define the folder path
folder_path = r'D:\BrownUnivercity\CS2470\final_proj\CS2470_final_project\data\test'
# load data
music,tag, align_length = get_music_data(folder_path, label_path)

In [4]:
print(f"music_shape:{music.shape}")

music_shape:(20, 10000, 88)


In [5]:
def one_hot_encode(labels, num_classes):
    one_hot_labels = np.zeros((len(labels), num_classes))
    for i, label in enumerate(labels):
        one_hot_labels[i, label - 1] = 1
    return one_hot_labels

## VAE 

In [6]:
class VAE(nn.Module):
    def __init__(self, input_size, hidden_dim, latent_size, label_size):
        super(VAE, self).__init__()
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.latent_size = latent_size
        self.label_size = label_size
        self.batch_sz = 5

        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )
        self.mu_layer = nn.Linear(256, latent_size)
        self.logvar_layer = nn.Linear(256, latent_size)

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(latent_size + label_size, 256),  # Adjust the output size to match (batch_sz, 10000, 88)
            nn.ReLU(),
            nn.Linear(256, 10000 * 88),  # Adjust the output size to match the desired output shape
            nn.Unflatten(1, (10000, 88))
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, label):
        # Encoding
        x = x.to(torch.float32)
        label = label.to(torch.float32)
        encoded = self.encoder(x)
        mu = self.mu_layer(encoded)
        logvar = self.logvar_layer(encoded)

        # Reparameterization trick
        z = self.reparameterize(mu, logvar)

        # Concatenate one-hot label with latent representation
        z_with_label = torch.cat((z, label), dim=1)

        # Decoding
        reconstructed = self.decoder(z_with_label)
        return reconstructed, mu, logvar




In [7]:
input_size = 10000 * 88  # Example input size
hidden_dim = 256
latent_size = 32
label_size = 4  # One-hot encoded label size
vae_model = VAE(input_size, hidden_dim, latent_size, label_size)

In [8]:
one_hot_labels = one_hot_encode(tag,4)
# print(one_hot_labels)
music = torch.tensor(music)
one_hot_labels = torch.tensor(one_hot_labels)

In [9]:
# Define your optimizer and loss function
optimizer = optim.Adam(vae_model.parameters(), lr=0.001)

# Reconstruction loss (MSE loss)
reconstruction_loss = nn.MSELoss()

# KL divergence loss
def kl_divergence_loss(mu, logvar):
    return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# Create a DataLoader for batch training
dataset = TensorDataset(music, one_hot_labels)
dataloader = DataLoader(dataset, shuffle=True)

# Number of training samples
total_samples = len(dataset)



In [14]:
# # Training loop
# epochs = 10
# for epoch in range(epochs):
#     # vae_model.train()
#     total_loss = 0.0
#     for batch_music, batch_label in dataloader:
#         batch_label = batch_label.to(torch.float32)
#         batch_music = batch_music.to(torch.float32)
#         optimizer.zero_grad()
#         reconstructed, mu, logvar = vae_model(batch_music, batch_label)
#         reconstructed = reconstructed.to(torch.float32)
#         mu = mu.to(torch.float32)
#         logvar = logvar.to(torch.float32)
#         # Compute reconstruction loss
#         recon_loss = reconstruction_loss(reconstructed, batch_music).to(torch.float32)
#         # Compute KL divergence loss
#         kl_loss = kl_divergence_loss(mu, logvar).to(torch.float32)
#         # Total loss is the sum of reconstruction loss and KL divergence loss
#         loss = recon_loss + kl_loss
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item() * len(batch_music)
#     average_loss = total_loss / total_samples
#     print(f"Epoch {epoch + 1}/{epochs}, Total Loss: {average_loss:.4f}")

In [15]:
epochs = 10

# Training loop
for epoch in range(epochs):
    total_loss = 0.0
    total_reconstruction_loss = 0.0
    total_kl_divergence_loss = 0.0

    for batch_idx, (music_batch, labels_batch) in enumerate(dataloader):
        optimizer.zero_grad()
        music_batch = music_batch.to(torch.float32)
        labels_batch = labels_batch.to(torch.float32)

        # Forward pass
        reconstructed_batch, mu, logvar = vae_model(music_batch, labels_batch)
        print(f"reconstructed_batch_shape:{reconstructed_batch.shape}")
        # print(f"music_batch:{music_batch}")
        reconstructed_batch = reconstructed_batch.to(torch.float32)
        mu = mu.to(torch.float32)
        logvar = logvar.to(torch.float32)

        # Compute reconstruction loss
        recon_loss = reconstruction_loss(reconstructed_batch, music_batch).to(torch.float32)

        # Compute KL divergence loss
        kld_loss = kl_divergence_loss(mu, logvar).to(torch.float32)

        # Total loss
        loss = recon_loss + kld_loss

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_reconstruction_loss += recon_loss.item()
        total_kl_divergence_loss += kld_loss.item()

    # Print statistics
    avg_loss = total_loss / total_samples
    avg_reconstruction_loss = total_reconstruction_loss / total_samples
    avg_kl_divergence_loss = total_kl_divergence_loss / total_samples
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}, '
          f'Reconstruction Loss: {avg_reconstruction_loss:.4f}, '
          f'KL Divergence Loss: {avg_kl_divergence_loss:.4f}')


reconstructed_batch_shape:torch.Size([3, 10000, 88])


KeyboardInterrupt: 